In [1]:
!pip install folium

     |████████████████████████████████| 95 kB 7.9 MB/s eta 0:00:011


In [2]:
import folium

map_y = folium.Map([37.52860, 126.93431]) #위도와 경도 입력
map_y #화면 출력

In [4]:
import folium

map_y = folium.Map([37.52860, 126.93431], zoom_start=15) #확대 및 축소 조절 zoom_start
map_y

In [6]:
import folium

lat, long = 37.52860, 126.93431
map_y = folium.Map([lat, long], zoom_start=15)
folium.Marker([lat, long]).add_to(map_y) #마커 표시
map_y

In [24]:
import folium

lat1, long1 = 37.52860, 126.93431 #여의도 한강공원
lat2, long2 = 37.52400, 126.91889 #여의도공원
lat3, long3 = 37.51865, 126.92041 #샛강생태공원

map_y = folium.Map([lat1, long1], zoom_start=15)
folium.Marker([lat1, long1], popup='여의도 한강공원', tooltip='여의도 한강공원', #팝업과 툴팁 둘다 가능
              icon=folium.Icon(color='red', icon='automobile', prefix='fa') #마커 색, 아이콘 변경 color, icon
             ).add_to(map_y)                                   #fontawesome의 아이콘 사용을 뜻하는 prefix='fa'사용
folium.Marker([lat2, long2], tooltip='여의도공원', #툴팁 
              icon=folium.Icon(color='blue', icon='home')
             ).add_to(map_y)
folium.Marker([lat3, long3], tooltip='샛강생태공원', #팝업
              icon=folium.Icon(color='purple', icon='ban', prefix='fa')
             ).add_to(map_y)
map_y

In [26]:
import folium

lat = [37.52860, 37.52400, 37.51865]
long = [126.93431, 126.91889, 126.92041]
names = ['여의도 한강공원', '여의도공원', '샛강생태공원']
icons = ['automobile', 'balance-scale', 'ban']
colors = ['red', 'blue', 'purple']

map_y = folium.Map([lat[0], long[0]], zoom_start=15) #초기 좌표 지정
for i in range(len(lat)):
    folium.Marker(
    [lat[i], long[i]], tooltip=names[i],
    icon = folium.Icon(color=colors[i], icon=icons[i], prefix='fa')
    ).add_to(map_y)
map_y

In [62]:
import csv
import operator
import folium

f1 = open('LOCAL_PEOPLE_DONG_201912.csv')
data = csv.reader(f1)
next(data)
data = list(data)

f2 = open('dong_code.csv')
code_data = csv.reader(f2)
next(code_data)
next(code_data)
code_data = list(code_data)

f3 = open('dong_location.csv', encoding='cp949') #인코딩이 다르므로 맥에선 cp949나 euc-kr을 해줘야 함
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

#인구 데이터의 데이터 타입 변환
for row in data:
    for i in range(1,32):
        if i <= 2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
            
#행정동코드 데이터의 데이터 타입 변환
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1])
    
#위치 정보 데이터의 데이터 타입 변환
for row in location_data:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1])

#행정동별 인구수 저장 딕셔너리    
dong_population = {}

#행정동별 총생활인구수 딕셔너리에 저장
for row in data:
    dong_code, p = row[2], row[3]
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p
    else:
        dong_population[dong_code] += p

#인구수 내림차순 정렬 후 TOP10 별도 저장
top10 = sorted(dong_population.items(), key=operator.itemgetter(1), reverse=True)[:10]

top10_code8 = []
for t in top10:
    top10_code8.append(t[0])

top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]
top10_long = [0 for i in range(10)]

#8자리 행정동코드에 해당하는 행정동명 찾기
for i in range(len(top10_code8)):
    for row in code_data:
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1]
            
#행정동명에 해당하는 위도와 경도를 리스트에 저장
    for row in location_data:
        if row[3] == top10_name[i]:
            top10_lat[i], top10_long[i] = row[-1], row[-2]

#행정동명으로 위치 정보를 찾지 못하는 경우 7자리 행정동코드로 찾기        
for i in range(len(top10_lat)):
    if top10_lat[i] == 0 or top10_long[i] == 0: #위도나 경도에 0인 요소가 있을 경우
        for row in code_data: #code_data에서 7자리 행정동코드를 찾아 dong7에 저장
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        for row in location_data: #location_data에서 7자리 행정동코드가 dong7인 행을 찾아 위도와 경도 찾기
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1], row[-2]

#행정동명 앞에 순위 표시
for i in range(len(top10_name)):
    top10_name[i] = str(i + 1)+'. '+top10_name[i]
    
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10):
    folium.Marker(
    [top10_lat[i], top10_long[i]], tooltip=top10_name[i],
    icon=folium.Icon(color='red', icon='star')
    ).add_to(map_top10)
map_top10

In [63]:
import csv
import operator
import folium

f1 = open('LOCAL_PEOPLE_DONG_201912.csv')
data = csv.reader(f1)
next(data)
data = list(data)

f2 = open('dong_code.csv')
code_data = csv.reader(f2)
next(code_data)
next(code_data)
code_data = list(code_data)

f3 = open('dong_location.csv', encoding='cp949') #인코딩이 다르므로 맥에선 cp949나 euc-kr을 해줘야 함
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

#인구 데이터의 데이터 타입 변환
for row in data:
    for i in range(1,32):
        if i <= 2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
            
#행정동코드 데이터의 데이터 타입 변환
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1])
    
#위치 정보 데이터의 데이터 타입 변환
for row in location_data:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1])

#행정동별 인구수 저장 딕셔너리    
dong_population = {}

#행정동별 총생활인구수 딕셔너리에 저장
for row in data:
    dong_code, p = row[2], row[3]
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p
    else:
        dong_population[dong_code] += p

#인구수 내림차순 정렬 후 TOP10 별도 저장
top10 = sorted(dong_population.items(), key=operator.itemgetter(1), reverse=True)[:10]

top10_code8 = []
for t in top10:
    top10_code8.append(t[0])

top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]
top10_long = [0 for i in range(10)]

#8자리 행정동코드에 해당하는 행정동명 찾기
for i in range(len(top10_code8)):
    for row in code_data:
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1]
            
#행정동명에 해당하는 위도와 경도를 리스트에 저장
    for row in location_data:
        if row[3] == top10_name[i]:
            top10_lat[i], top10_long[i] = row[-1], row[-2]

#행정동명으로 위치 정보를 찾지 못하는 경우 7자리 행정동코드로 찾기        
for i in range(len(top10_lat)):
    if top10_lat[i] == 0 or top10_long[i] == 0: #위도나 경도에 0인 요소가 있을 경우
        for row in code_data: #code_data에서 7자리 행정동코드를 찾아 dong7에 저장
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        for row in location_data: #location_data에서 7자리 행정동코드가 dong7인 행을 찾아 위도와 경도 찾기
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1], row[-2]

#행정동명 앞에 순위 표시
for i in range(len(top10_name)):
    top10_name[i] = str(i + 1)+'. '+top10_name[i]
    
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10):
    if i < 5:
        folium.Marker(
        [top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='darkblue', icon='male', prefix='fa')
        ).add_to(map_top10)
    else:
        folium.Marker(
        [top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='lightblue', icon='male', prefix='fa')
        ).add_to(map_top10)
map_top10

# 12장 실습

### 1번

In [65]:
import folium

map_y = folium.Map([35.15870, 129.16047], zoom_start=11)
folium.Marker([35.15870, 129.16047], icon=folium.Icon(icon='hand-o-down', prefix='fa')).add_to(map_y)
map_y

### 2번

In [69]:
import folium

map_lat = [35.15870, 35.23360, 35.17357]
map_long = [129.16047, 129.08309, 128.94632]

map_y = folium.Map([map_lat[0], map_long[0]])

folium.Marker([map_lat[0], map_long[0]], tooltip='해운대 해수욕장', icon=folium.Icon(icon='hand-o-down', prefix='fa')
             ).add_to(map_y)
folium.Marker([map_lat[1], map_long[1]], tooltip='부산대학교', icon=folium.Icon(icon='university', prefix='fa')
             ).add_to(map_y)
folium.Marker([map_lat[2], map_long[2]], tooltip='김해국제공항', icon=folium.Icon(icon='plane', prefix='fa')
             ).add_to(map_y)
map_y